<a href="https://colab.research.google.com/github/rajdeepbanerjee-git/JNCLectures_Intro_to_ML/blob/main/Week8/Week8_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.special import expit  # Sigmoid function

import warnings
warnings.filterwarnings("ignore")  # Suppress all warnings


#### logistic regression using gradient descent

In [33]:
def compute_gradient(beta, X, y):
    p = expit(X @ beta)
    return X.T @ (p - y)  # Gradient of log-likelihood

def logistic_regression(X, y, lr=0.01, epochs=1000):
    m, n = X.shape
    beta = np.zeros(n)  # Initialize coefficients

    for _ in range(epochs):
        gradient = compute_gradient(beta, X, y)
        beta -= lr * gradient  # Gradient descent update

    return beta

In [34]:
# Example usage
np.random.seed(42)
X = np.c_[np.ones(100), np.random.rand(100, 2)]  # Add intercept term, X has two features randomly generated
y = (X[:, 1] + X[:, 2] > 1).astype(int)  # Generate synthetic labels: y = 1 is sum of the two features being > 1
print(X, y)

[[1.         0.37454012 0.95071431]
 [1.         0.73199394 0.59865848]
 [1.         0.15601864 0.15599452]
 [1.         0.05808361 0.86617615]
 [1.         0.60111501 0.70807258]
 [1.         0.02058449 0.96990985]
 [1.         0.83244264 0.21233911]
 [1.         0.18182497 0.18340451]
 [1.         0.30424224 0.52475643]
 [1.         0.43194502 0.29122914]
 [1.         0.61185289 0.13949386]
 [1.         0.29214465 0.36636184]
 [1.         0.45606998 0.78517596]
 [1.         0.19967378 0.51423444]
 [1.         0.59241457 0.04645041]
 [1.         0.60754485 0.17052412]
 [1.         0.06505159 0.94888554]
 [1.         0.96563203 0.80839735]
 [1.         0.30461377 0.09767211]
 [1.         0.68423303 0.44015249]
 [1.         0.12203823 0.49517691]
 [1.         0.03438852 0.9093204 ]
 [1.         0.25877998 0.66252228]
 [1.         0.31171108 0.52006802]
 [1.         0.54671028 0.18485446]
 [1.         0.96958463 0.77513282]
 [1.         0.93949894 0.89482735]
 [1.         0.59789998 0.92

In [35]:
# calculate beta by running gradient descent
beta = logistic_regression(X, y)
print("Estimated coefficients:", beta)

Estimated coefficients: [-10.17510762  10.7396318    9.20252423]


In [36]:
# predict
y_pred = expit(X @ beta)
results_df = pd.DataFrame({'Actual': y, 'Predicted': y_pred})
results_df

,Actual,Predicted
0,1,0.930622
1,1,0.960661
2,0,0.000855
3,0,0.170752
4,1,0.942497
...,...,...
95,0,0.285250
96,1,0.995137
97,0,0.034876
98,1,0.997205


In [37]:
# confusion matric based on threshold probability
threshold_proba = 0.5
results_df['predicted_label'] = results_df['Predicted'].apply(lambda x: 1 if x >= threshold_proba else 0)
confusion_matrix = pd.crosstab(results_df['Actual'], results_df['predicted_label'], rownames=['Actual'], colnames=['Predicted'])
confusion_matrix

Predicted,0,1
Actual,,
0,56,1
1,3,40


#### The Newton-Raphson method

In [31]:
def newton_raphson_logistic_regression(X, y, tol=1e-6, max_iter=100):

    # Initialize beta (n_features,)
    beta = np.zeros(X.shape[1])

    for iteration in range(max_iter):
        # Compute the predicted probabilities using the sigmoid function
        p = expit(X @ beta)  # shape (n_samples,)

        # Compute the gradient: X^T (y - p)
        gradient = X.T @ (y - p)

        # Build the weight matrix W as a diagonal matrix with entries p*(1-p)
        W = np.diag(p * (1 - p))

        # Compute the Hessian matrix: H = -X^T W X. We use H_inv = (X^T W X)^{-1} directly.
        Hessian = -X.T @ W @ X

        # Newton-Raphson update: beta_new = beta + (X^T W X)^{-1} X^T (y - p)
        delta = np.linalg.inv(Hessian) @ gradient

        # Update beta
        beta_new = beta - delta

        # Check convergence (using L2 norm of the change)
        if np.linalg.norm(beta_new - beta, ord=2) < tol:
            beta = beta_new
            print(f'Convergence reached after {iteration+1} iterations.')
            break

        beta = beta_new

    return beta





In [46]:
# Compute beta using Newton-Raphson for logistic regression
beta_newton = newton_raphson_logistic_regression(X, y, tol=1e-3, max_iter=10)
print("Estimated coefficients (beta) using Newton-Raphson:")
print(beta_newton)

Estimated coefficients (beta) using Newton-Raphson:
[-250.68995337  251.57845749  250.98674829]


In [47]:
# predict
y_pred_nr = expit(X @ beta_newton)
results_df_nr = pd.DataFrame({'Actual': y, 'Predicted': y_pred_nr})
results_df_nr

,Actual,Predicted
0,1,1.000000e+00
1,1,1.000000e+00
2,0,1.502928e-75
3,0,7.726317e-09
4,1,1.000000e+00
...,...,...
95,0,1.112499e-01
96,1,1.000000e+00
97,0,1.711735e-34
98,1,1.000000e+00


In [48]:
# confusion matric based on threshold probability
threshold_proba = 0.5
results_df_nr['predicted_label'] = results_df_nr['Predicted'].apply(lambda x: 1 if x >= threshold_proba else 0)
confusion_matrix_nr = pd.crosstab(results_df_nr['Actual'], results_df_nr['predicted_label'], rownames=['Actual'], colnames=['Predicted'])
confusion_matrix_nr

Predicted,0,1
Actual,,
0,57,0
1,0,43


For this example you reached a better result with much less number of steps - so **convergence is much much faster!**

Now, if you don't want to do it from scratch you can also use the scipy optimize library to do the optimization job for you!

You can even choose other optimization methods! For example, here we use bfgs which both better convergence than gradient descent and computationally less expensive than Newton Raphson.

In [43]:
# Log-likelihood function (to be maximized)
def log_likelihood(beta, X, y):
    p = expit(X @ beta)
    return -np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))  # Negative for minimization

# Gradient of log-likelihood
def gradient(beta, X, y):
    p = expit(X @ beta)
    return X.T @ (p - y)  # Gradient of negative log-likelihood

# Fit logistic regression using BFGS
def logistic_regression_bfgs(X, y):
    beta_init = np.zeros(X.shape[1])  # Initialize coefficients to zero
    result = minimize(fun=log_likelihood,
                      x0=beta_init,
                      args=(X, y),
                      jac=gradient,  # Jacobian (gradient)
                      method='BFGS')  # Use BFGS method
    return result.x  # Optimized coefficients

# Solve logistic regression using BFGS
beta_opt = logistic_regression_bfgs(X, y)
print("Optimized coefficients using BFGS:", beta_opt)


Optimized coefficients using BFGS: [-47.84985556  46.94303161  47.61092446]


In [45]:
# predict
y_pred_bfgs = expit(X @ beta_opt)
results_df_bfgs = pd.DataFrame({'Actual': y, 'Predicted': y_pred_nr})
results_df_bfgs

,Actual,Predicted
0,1,1.000000e+00
1,1,1.000000e+00
2,0,1.502928e-75
3,0,7.726317e-09
4,1,1.000000e+00
...,...,...
95,0,1.112499e-01
96,1,1.000000e+00
97,0,1.711735e-34
98,1,1.000000e+00


In [51]:
# confusion matric based on threshold probability
threshold_proba = 0.5
results_df_bfgs['predicted_label'] = results_df_bfgs['Predicted'].apply(lambda x: 1 if x >= threshold_proba else 0)
confusion_matrix_bfgs = pd.crosstab(results_df_bfgs['Actual'], results_df_bfgs['predicted_label'], rownames=['Actual'], colnames=['Predicted'])
confusion_matrix_bfgs

Predicted,0,1
Actual,,
0,57,0
1,0,43


**Summary:**

| Method            | Convergence Speed  | Computational Cost            | Suitable for Large Data          |
|------------------|------------------|------------------------------|--------------------------------|
| Newton-Raphson   | Fast (quadratic)  | High (Hessian inversion)      | No (Expensive)                 |
| Gradient Descent | Slow (linear)     | Low (only gradient)          | Yes (Scales well)               |
| BFGS            | Fast & stable     | Moderate (Hessian approximation) | Yes (L-BFGS for large data)     |
